# Draw on Video
#### This notebook draws SIFT feature locations into a video.
#### Only features matched across consecutive frames are drawn. 

In [ ]:
import skvideo.io  
import matplotlib.pyplot as plt

# Handles relative import
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: sys.path.append(dir1)

import cv2
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import match as match_lib 

from sift import detect_sift_features

## Detect and draw

In [ ]:
videodata = skvideo.io.vread("../videos/desk.mp4")  
cmap = matplotlib.cm.get_cmap('autumn_r')

anot_vid = list()

prev_features = None 
features = None
color = (0, 255, 0)
frac_width = videodata.shape[2] / 8
frac_width = frac_width / 6
nr_frames = videodata.shape[0]

for idx, frame in enumerate(videodata):
    if not idx % 10: 
        print(f'At frame {idx} out of {nr_frames}')
    img_col = cv2.resize(frame, None, fx=0.1, fy=0.1, interpolation=cv2.INTER_LINEAR)
    img = cv2.cvtColor(img_col, cv2.COLOR_RGB2GRAY)
    img = cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    features = detect_sift_features(img)
    
    if prev_features:
        matches = match_lib.match_sift_features(prev_features, features)
        for feat1, feat2 in matches: 
            if np.linalg.norm(feat2.absolute_coordinate - feat1.absolute_coordinate) < frac_width: 
                s, y, x = feat2.absolute_coordinate.round().astype(int)
                p_s, p_y, p_x = feat1.absolute_coordinate.round().astype(int)
                img_col = cv2.circle(img_col, (x, y), 2, color=color, thickness=-1)
                img_col = cv2.line(img_col, (p_x, p_y), (x, y), color=color, thickness=1)

    prev_features = features
    anot_vid.append(img_col)


## Write To Disk

In [ ]:
anot_vid = np.array(anot_vid).astype(np.uint8)
repeated = anot_vid.repeat(5, 0)
skvideo.io.vwrite("desk_with_features.mp4", anot_vid)